In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn import set_config
set_config(display='diagram')   
# displays HTML representation in a jupyter context

### 1.Import data

In [245]:
data = pd.read_csv('../data/loan/train.csv')

data = data.drop('Loan_ID', axis=1)
data['Dependents'] = data['Dependents'].replace('3+', '3')

X, y = data.drop('Loan_Status', axis=1), data.pop('Loan_Status')

X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

In [3]:
ct_fill = make_column_transformer(
    (SimpleImputer(strategy="most_frequent"),make_column_selector(dtype_exclude=np.number)),
    (SimpleImputer(strategy="mean"), make_column_selector(dtype_include=np.number))
)

In [4]:
ct_fill

ColumnTransformer(transformers=[('simpleimputer-1',
                                 SimpleImputer(strategy='most_frequent'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001B41EF77760>),
                                ('simpleimputer-2', SimpleImputer(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001B41EF77400>)])

In [247]:
X_train = pd.DataFrame(ct_fill.fit_transform(X_train)).infer_objects()
X_test = pd.DataFrame(ct_fill.transform(X_test)).infer_objects()

y_train = y_train.map({'Y':1, 'N':0})
y_test = y_test.map({'Y':1, 'N':0})

In [248]:
cat_features = X_train.select_dtypes('object')
cat_features_idx = [X_train.columns.get_loc(col) for col in cat_features]

### 2. Install LightGBM

In [249]:
#%pip install catboost
from catboost import CatBoostClassifier

### 3. Build a Model

In [250]:
model = CatBoostClassifier(verbose=False)


### 3. Evaluate model

In [251]:
model.fit(X_train, y_train, cat_features=cat_features_idx, eval_set=(X_test, y_test))
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8108695652173913, 0.8116883116883117)